# Pytorch-GRU-LSTM
##
1. 

In [1]:
import pandas as pd
import torch
import numpy as np
from torch import nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from sklearn.preprocessing import MinMaxScaler
from torch.autograd import Variable

In [2]:
# 全局参数
type_map = {'train':0, 'val':1, 'test':2}
input_path_dir = 'C:/ZhangLI/Codes/DataSet/kdd-cup/'
input_path_dir = 'E:/Dataset/kdd/'
file_name = 'sdwpf_baidukddcup2022_full.csv'
cols = ['Wspd', 'Wdir', 'Etmp', 'Itmp', 'Ndir', 'Pab1', 'Pab2', 'Pab3', 'Prtv', 'Patv']
lable = 'Patv'

train_size = 24 * 6 * 153
val_size = 24 * 6 * 16
test_size = 24 * 6 * 15

In [3]:
data_raw = pd.read_csv(f'{input_path_dir}{file_name}')

In [4]:
# 读取数据的类
class WPFDataset(Dataset):
    """
    Desc: Data
        ...
    """
    def __init__(self, data):
        """
        Desc: 
        Input: DataSet
        """
        self.data_set = data
        self.train_size = train_size
        self.val_size = val_size
        self.test_size = test_size
        self._start = [0, 153*144-144, 169*144-144]  # [0, 21888, 24192]
        self._end = [153*144, 169*144, 184*144]  # [22032, 24336, 26496]
        
    def get_data(self, flag='train'):
        """
        Desc: 
        Input: 
            train: train data
            val: val data
            test: test data
        """
        if flag == 'train':
            clip_start = self._start[type_map[flag]]
            clip_end = self._end[type_map[flag]]
        elif flag == 'val':
            clip_start = self._start[type_map[flag]]
            clip_end = self._end[type_map[flag]]
        self._data_set = self.data_set[clip_start:clip_end]
        
        
        
        
    def __getitem__(self, index):
        """
        Desc:
        
        """
        s_begin = index
        s_end = index + 30de0  # 使用200个点预测288个点
        e_begin = s_end
        e_end = s_end + 288
        seq_x = self._data_set[s_begin:s_end]
        seq_y = self._data_set[e_begin:e_end]
        return seq_x, seq_y
    
    def __len__(self):
        """
        Desc:
        
        """
        #return int((len(self._data_set)-200)/288)
        return int(len(self._data_set) - 300 - 288 + 1)  # 数据的长度

ds = WPFDataset(data_raw[cols].values) 
ds.get_data()
dl = DataLoader(ds, batch_size=32, shuffle=True, drop_last=True)


# WPFDataset[2] 使用索引的方式获取数据时，会自动调用 getitem 方法
count = 1
for i in dl:
    count += 1
count

671

In [5]:
32 * 386
int((len(data_raw)-200)/288)

12327

In [6]:
class RnnModel(nn.Module):
    """
    Desc: define model
    """
    def __init__(self):
        """
        Desc: init class
        """
        super(RnnModel, self).__init__()
        self.input_l = 10  # 特征的个数
        self.output_l = 288  # 预测的长度
        self.hidden_l = 48  # 隐藏层的个数
        self.layer_l = 1
        
        self.lstm = nn.GRU(input_size=self.input_l, hidden_size=self.hidden_l, num_layers=1, batch_first=True, dropout=0.25)
        self.fn = nn.Linear(in_features=self.hidden_l, out_features=1)
    
    def forward(self, x):
        """
        Desc: forward
        Input: x[batch_size, seq_len(time_step), feature_num]
            h0[bi*num_layer, batch_size, hidden_size]
        """
        h0 = Variable(torch.zeros(1, x.size(0), self.hidden_l))
        # TODO input and output
        h_out, hn = self.lstm(x, h0)  # 
        # print(h_out.size())
        out = self.fn(h_out)
        # print(out.size())
        
        return out

In [10]:
def trian_and_val():
    # 获取模型
    model = RnnModel()
    # 数据处理，获取数据集
    data_raw[cols] = data_raw[cols].fillna(0)
    scaler = MinMaxScaler() # 进行实例化
    scaler = scaler.fit(data_raw[cols]) # 生成min()和max()
    data_tran = scaler.transform(data_raw[cols])
    ds = WPFDataset(data_tran)   
    dl = DataLoader(ds, batch_size=32, shuffle=True, drop_last=True)
    # 定义参数
    model_optim = torch.optim.Adam(params=model.parameters())
    criterion = torch.nn.MSELoss()
    # 训练
    epoches = 3
    for i in range(epoches):
        model.train()
        train_loss = []
        for x, y in dl:
            x = x.to(torch.float32)
            y = y[:, -288:, -1:].type(torch.float64)
            #print(y.size())
            
            y_ = model(x)
            y_ = y_[..., -288:, -1:].type(torch.float64)  # 这里有点瞎搞的意思
            #print(y_.size())
            loss = criterion(y, y_)
            train_loss.append(loss.item())
            loss.backward()
            # model_optim.maximize(loss)
            model_optim.step()
            model_optim.zero_grad()
            # print(loss)
        is_debug = True
        if is_debug:
            train_loss = np.average(train_loss)
            epoch_end_time = time.time()
            print("Epoch: {}, \nTrain Loss: {}, \nValidation Loss: {}".format(i, train_loss, train_loss))
            # print("Elapsed time for epoch-{}: {}".format(epoch, epoch_end_time - epoch_start_time))
    # 停止训练
    # 模型保存
# XXDataSet 创建数据类，数据的预测，数据的加载，数据的处理
    # getitem  # 这些都是内置的方法
    # len
    # process
    # get_data # 留一个外置接口API就可以了
# Models 单纯的定义类
    # forward
# Experience 创建模型类，定义训练过程 参数，训练，停止 模型保存  [进化成专家的过程]
    # seeding
    # optim
    # critie
    # process one 
    # stop
    # save
    # get model -- statis
# trian and val 每一个模型的真实训练
    # train
    # val
# configs
    # -- perpare
    # -- consant
    # -- 
# logs
# checkpoint
# utils
# predict
trian_and_val()        

d:\installsoftware\python365\lib\site-packages\torch\nn\modules\rnn.py:65: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.25 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


tensor(0.1679, dtype=torch.float64, grad_fn=<MseLossBackward>)
tensor(0.2071, dtype=torch.float64, grad_fn=<MseLossBackward>)
tensor(0.1622, dtype=torch.float64, grad_fn=<MseLossBackward>)
tensor(0.1436, dtype=torch.float64, grad_fn=<MseLossBackward>)
tensor(0.1274, dtype=torch.float64, grad_fn=<MseLossBackward>)
tensor(0.1703, dtype=torch.float64, grad_fn=<MseLossBackward>)
tensor(0.1200, dtype=torch.float64, grad_fn=<MseLossBackward>)
tensor(0.1138, dtype=torch.float64, grad_fn=<MseLossBackward>)
tensor(0.0851, dtype=torch.float64, grad_fn=<MseLossBackward>)
tensor(0.1219, dtype=torch.float64, grad_fn=<MseLossBackward>)
tensor(0.1146, dtype=torch.float64, grad_fn=<MseLossBackward>)
tensor(0.1147, dtype=torch.float64, grad_fn=<MseLossBackward>)
tensor(0.1355, dtype=torch.float64, grad_fn=<MseLossBackward>)
tensor(0.1139, dtype=torch.float64, grad_fn=<MseLossBackward>)
tensor(0.1189, dtype=torch.float64, grad_fn=<MseLossBackward>)
tensor(0.1188, dtype=torch.float64, grad_fn=<MseLossBac

KeyboardInterrupt: 

In [91]:
y

NameError: name 'y' is not defined